Raw data


In [ ]:
from pyspark.sql.functions import * 

In [ ]:
# Azure Event Hub Configuration
event_hub_namespace = "hospital-analytics-eh-namespace.servicebus.windows.net"
event_hub_name="hospital-analytics-eh"  
event_hub_conn_str = dbutils.secrets.get(scope = "hospitalanalyticsvaultscope", key = "eventhub-connection")

In [ ]:
# reading stream from event hub
raw_df = (spark.readStream
          .format("kafka")
          .options(**kafka_options)
          .load()
          )
#Cast data to json (converting binary data in raw json string)
json_df = raw_df.selectExpr("CAST(value AS STRING) as raw_json")

#ADLS configuration 
spark.conf.set(
  "fs.azure.account.key.hospitalstorageac.dfs.core.windows.net",
  dbutils.secrets.get(scope = "hospitalanalyticsvaultscope", key = "storage-connection")
)

bronze_path = "abfss://bronze@hospitalstorageac.dfs.core.windows.net/patient_flow"

#Write stream to bronze
(
    json_df
    .writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", "dbfs:/mnt/bronze/_checkpoints/patient_flow")
    .start(bronze_path)
)

In [ ]:
display(spark.read.format("delta").load(bronze_path))